### Modeling of manually-derived strains
Computer-aided cell factory engineering was quite recently introduced to metabolic engineering, as a high throuput analysis tool to identify gene modifications and simulate the chaotic metabolic processes incide the cells. However, metabolic engineering has a longer history, and there have been a lot of efforts to achieve high production of desired products, by lowering or increasing the expression of intiitive and logically derived gene targets.
In this section, we are going to evaluate some popular manual derived changes through model simulations. Those are the downregulation of squalene synthase (<i>ERG9</i>), the upregulation of 3-hydroxy-3-methylglutaryl-coenzyme A reductase (<i>HMG1</i>) and the 